In [1]:
import sympy as sp
from sympy import abc

In [2]:
from scipy.integrate import quad
from scipy.special import dawsn
import numpy as np
from functools import lru_cache
import scipy.optimize as opt
import numdifftools as nd

# Ферромагнетик

## Выражение для $Z^{(2)}$

### Определения

In [3]:
class SigmaSymbol(sp.Symbol):
    def _latex(self, printer, exp=None):
        if exp is None:
            return self.name
        else:
            exp = printer.doprint(exp)
            return f"{self.name}^{exp}"

    def _eval_power(self, expt):
        if expt.is_integer:
            d, m = divmod(expt, 2)
            return self**m


SIGMAS = [-1, 1]

In [4]:
class F2Function(sp.Function):
    def _latex(self, printer, exp=None):
        arg = printer.doprint(self.args[0])
        if exp is None:
            return "F^{(2)}"+f"\\left({arg}\\right)"
        else:
            exp = printer.doprint(exp)
            return f"\\left({self._latex(printer)}\\right)^{exp}"

    def fdiff(self, argindex=1):
        return 1-2*self.args[0]*F2Function(self.args[0])

    def _eval_rewrite(self, rule, args, **hints):
        x, = args
        if rule == sp.erfi:
            return sp.sqrt(sp.pi)/2*sp.exp(-x**2)*sp.erfi(x)

    @classmethod
    def eval(cls, z):
        if z.could_extract_minus_sign():
            return -cls(-z)
        if z == 0:
            return 0

### Аргументы

In [5]:
h, l = sp.symbols("h \\lambda", positive=True)
s1, s2 = SigmaSymbol("\\sigma_1"), SigmaSymbol("\\sigma_2")
t = sp.Symbol("t")

In [6]:
arg_exp = t**2/(2*l) + s1*t
bound = h+s2*l

arg_exp = arg_exp.replace(t, t-s1*l).expand()
bound += s1*l

arg_exp = arg_exp.replace(t, t*sp.sqrt(2*l))
bound /= sp.sqrt(2*l)

display(arg_exp)
display(bound)

-\lambda/2 + t**2

sqrt(2)*(\sigma_1*\lambda + \sigma_2*\lambda + h)/(2*sqrt(\lambda))

In [7]:
expr = bound**2 + (arg_exp - t**2)
expr = expr.expand()
expr -= (h**2+l**2)/(2*l)
expr = expr.expand()

display(expr)

\sigma_1*\sigma_2*\lambda + \sigma_1*h + \sigma_2*h

### Производные $\cal{F}^{(2)}$

In [8]:
expr = s1*s2*sp.exp((s1+s2)*h+s1*s2*l)*F2Function((h+(s1+s2)*l)/sp.sqrt(2*l))
expr = expr.diff(h).expand()
display(expr)

-\sigma_1*\sigma_2*h*exp(\sigma_1*h)*exp(\sigma_2*h)*exp(\sigma_1*\sigma_2*\lambda)*F2Function(sqrt(2)*\sigma_1*sqrt(\lambda)/2 + sqrt(2)*\sigma_2*sqrt(\lambda)/2 + sqrt(2)*h/(2*sqrt(\lambda)))/\lambda + sqrt(2)*\sigma_1*\sigma_2*exp(\sigma_1*h)*exp(\sigma_2*h)*exp(\sigma_1*\sigma_2*\lambda)/(2*sqrt(\lambda))

In [9]:
expr = s1*s2*sp.exp((s1+s2)*h+s1*s2*l)*F2Function((h+(s1+s2)*l)/sp.sqrt(2*l))
expr = expr.diff(l).expand()
display(expr)

-\sigma_1*\sigma_2*exp(\sigma_1*h)*exp(\sigma_2*h)*exp(\sigma_1*\sigma_2*\lambda)*F2Function(sqrt(2)*\sigma_1*sqrt(\lambda)/2 + sqrt(2)*\sigma_2*sqrt(\lambda)/2 + sqrt(2)*h/(2*sqrt(\lambda))) + \sigma_1*\sigma_2*h**2*exp(\sigma_1*h)*exp(\sigma_2*h)*exp(\sigma_1*\sigma_2*\lambda)*F2Function(sqrt(2)*\sigma_1*sqrt(\lambda)/2 + sqrt(2)*\sigma_2*sqrt(\lambda)/2 + sqrt(2)*h/(2*sqrt(\lambda)))/(2*\lambda**2) - sqrt(2)*\sigma_1*\sigma_2*h*exp(\sigma_1*h)*exp(\sigma_2*h)*exp(\sigma_1*\sigma_2*\lambda)/(4*\lambda**(3/2)) + sqrt(2)*\sigma_1*exp(\sigma_1*h)*exp(\sigma_2*h)*exp(\sigma_1*\sigma_2*\lambda)/(4*sqrt(\lambda)) + sqrt(2)*\sigma_2*exp(\sigma_1*h)*exp(\sigma_2*h)*exp(\sigma_1*\sigma_2*\lambda)/(4*sqrt(\lambda))

In [10]:
class EFunction(sp.Function):
    """
    args[0], args[1] -- h, l
    args[2]: {0, 1} -- есть ли sigma_1 в множителе внутри суммы
    args[3]: {0, 1} -- есть ли sigma_2 в множителе внутри суммы
    EFunction(h, l, 1, 1).diff(h) -> 2*EFunction(h, l, 1, 0)
    """

    def _latex(self, printer, exp=None):
        args = printer.doprint(self.args[0]), printer.doprint(self.args[1])
        args = f"\\left({args[0]}, {args[1]}\\right)"

        indexes = ""
        for i, item in enumerate(self.args[2:]):
            if item:
                indexes += str(i+1)
        name = "{\\cal E}_{"+indexes+"}"

        if exp is None:
            return name+args
        else:
            exp = printer.doprint(exp)
            return f"{name}^{exp}"+args

    def fdiff(self, argindex=1):
        if argindex == 1:
            args = self.args[0], self.args[1]
            term1 = EFunction(
                *args,
                int(not (self.args[2] and True)),
                self.args[3]
            )
            term2 = EFunction(
                *args,
                self.args[2],
                int(not (self.args[3] and True))
            )
            return term1 + term2

        if argindex == 2:
            args = self.args[0], self.args[1]
            return EFunction(
                *args,
                int(not (self.args[2] and True)),
                int(not (self.args[3] and True))
            )

    @classmethod
    def eval(cls, *args):
        if (not args[2]) and args[3]:
            return EFunction(args[0], args[1], 1, 0)

    def _eval_rewrite(self, rule, args, **hints):
        h, l, arg_s1, arg_s2 = args
        if rule == sp.exp:
            result = 0
            for s1 in SIGMAS:
                for s2 in SIGMAS:
                    term = sp.exp((s1+s2)*h+s1*s2*l)
                    if arg_s1:
                        term *= s1
                    if arg_s2:
                        term *= s2
                    result += term
            return result

In [11]:
class cF2Function(sp.Function):
    def _latex(self, printer, exp=None):
        args = printer.doprint(self.args[0]), printer.doprint(self.args[1])
        if exp is None:
            return "{\\cal F}^{(2)}"+f"\\left({args[0]},{args[1]}\\right)"
        else:
            exp = printer.doprint(exp)
            return f"\\left({self._latex(printer)}\\right)^{exp}"

    def fdiff(self, argindex=1):
        h, l = self.args
        if argindex == 1:
            return -h/l*cF2Function(h, l) + 1/sp.sqrt(2*l)*EFunction(h, l, 1, 1)

        if argindex == 2:
            term1 = (h**2/(2*l**2)-1)*cF2Function(h, l)
            term2 = -h/(2*l*sp.sqrt(2*l))*EFunction(h, l, 1, 1)
            term3 = 1/sp.sqrt(2*l)*EFunction(h, l, 1, 0)
            return term1 + term2 + term3

    def _eval_rewrite(self, rule, args, **hints):
        h, l = args
        if rule == sp.exp:
            result = 0
            for s1 in SIGMAS:
                for s2 in SIGMAS:
                    term = sp.exp((s1+s2)*h+s1*s2*l)
                    term *= F2Function((h+(s1+s2)*l)/sp.sqrt(2*l))
                    term *= s1*s2
                    result += term
            return result

In [12]:
display(EFunction(h, l, 1, 1).diff(h))

display(EFunction(h, l, 1, 1).rewrite(sp.exp).diff(h))
display(EFunction(h, l, 1, 0).rewrite(sp.exp))

2*EFunction(h, \lambda, 1, 0)

-2*exp(\lambda - 2*h) + 2*exp(\lambda + 2*h)

-exp(\lambda - 2*h) + exp(\lambda + 2*h)

In [13]:
display(cF2Function(h, l).diff(l))

display(cF2Function(h, l).rewrite(sp.exp))

(-1 + h**2/(2*\lambda**2))*cF2Function(h, \lambda) + sqrt(2)*EFunction(h, \lambda, 1, 0)/(2*sqrt(\lambda)) - sqrt(2)*h*EFunction(h, \lambda, 1, 1)/(4*\lambda**(3/2))

exp(\lambda - 2*h)*F2Function(sqrt(2)*(-2*\lambda + h)/(2*sqrt(\lambda))) + exp(\lambda + 2*h)*F2Function(sqrt(2)*(2*\lambda + h)/(2*sqrt(\lambda))) - 2*exp(-\lambda)*F2Function(sqrt(2)*h/(2*sqrt(\lambda)))

In [14]:
for s1 in [1, 0]:
    for s2 in [1, 0]:
        # print(f"s1 = {s1}, s2 = {s2}")
        display(EFunction(h, l, s1, s2).rewrite(sp.exp))

exp(\lambda - 2*h) + exp(\lambda + 2*h) - 2*exp(-\lambda)

-exp(\lambda - 2*h) + exp(\lambda + 2*h)

-exp(\lambda - 2*h) + exp(\lambda + 2*h)

exp(\lambda - 2*h) + exp(\lambda + 2*h) + 2*exp(-\lambda)

### Численный расчет

#### Определения

Расчет сумм по $\sigma_{1,2}$ разбит на два этапа.

**1**. Расчет массива значений в зависимости от $\sigma_1$ и $\sigma_2$.

**2**. Суммирование значений с множителем, зависящем от $\sigma_{1,2}$.

In [15]:
def f_F2Function(x):
    return dawsn(x)

In [16]:
@lru_cache(maxsize=1)
def _Z2_E_terms(h, l):
    def func(s1, s2): return np.exp((s1+s2)*h+s1*s2*l)
    return np.asarray([[func(s1, s2) for s2 in SIGMAS] for s1 in SIGMAS])

@lru_cache(maxsize=1)
def _Z2_E_terms_norm(h, l):
    def func(s1, s2): return np.exp((s1+s2)*h+s1*s2*l-2*h-l)
    return np.asarray([[func(s1, s2) for s2 in SIGMAS] for s1 in SIGMAS])


@lru_cache(maxsize=1)
def _Z2_F_terms(h, l):
    def func(s1, s2): return f_F2Function((h+(s1+s2)*l)/np.sqrt(2*l))
    return np.asarray([[func(s1, s2) for s2 in SIGMAS] for s1 in SIGMAS])

In [17]:
def _Z2_sum_terms(terms, sigmas_func):
    result = 0
    for s1 in range(2):
        for s2 in range(2):
            result += sigmas_func(SIGMAS[s1], SIGMAS[s2])*terms[s1, s2]

    return result

In [18]:
def f_cF2Function(h, l):
    _terms = _Z2_E_terms(h, l)*_Z2_F_terms(h, l)
    def _sigmas_func_12(s1, s2): return s1*s2
    return _Z2_sum_terms(_terms, _sigmas_func_12)


def f_cF2Function_norm(h, l):
    _terms = _Z2_E_terms_norm(h, l)*_Z2_F_terms(h, l)
    def _sigmas_func_12(s1, s2): return s1*s2
    return _Z2_sum_terms(_terms, _sigmas_func_12)


def v_cF2Function(x): return f_cF2Function(*x)

In [19]:
def f_Z1(p):
    return 4*np.pi/p*np.sinh(p)


def f_Z2(h, l):
    return np.sqrt(2)*(2*np.pi)**2/(h*np.sqrt(l))*f_cF2Function(h, l)

In [20]:
def raw_Z2(h, l):
    def _sqrt(x): return np.sqrt(h**2+l**2+2*h*l*x)
    def func(x): return np.exp(h*x)*f_Z1(_sqrt(x))
    return 2*np.pi*quad(func, -1, 1)[0]


def v_raw_Z2(x): return raw_Z2(*x)

In [21]:
def f_EFunction(h, l, arg_s1, arg_s2):
    def _sigmas_func(s1, s2):
        result = 1
        if arg_s1:
            result *= s1
        if arg_s2:
            result *= s2
        return result
    return _Z2_sum_terms(_Z2_E_terms(h, l), _sigmas_func)


def f_EFunction_norm(h, l, arg_s1, arg_s2):
    def _sigmas_func(s1, s2):
        result = 1
        if arg_s1:
            result *= s1
        if arg_s2:
            result *= s2
        return result
    return _Z2_sum_terms(_Z2_E_terms_norm(h, l), _sigmas_func)


module_dictionary = {"cF2Function": f_cF2Function, "EFunction": f_EFunction, "F2Function": f_F2Function}
module_dictionary_norm = {"cF2Function": f_cF2Function_norm, "EFunction": f_EFunction_norm, "F2Function": f_F2Function}

#### Проверка

In [22]:
args = 1, 1.5
print(raw_Z2(*args))
print(f_Z2(*args))

350.1775720941783
350.1775720941784


In [23]:
f_cF2_diff_h = sp.lambdify((h, l), cF2Function(h, l).diff(h), [module_dictionary, "scipy"])
f_cF2_diff_l = sp.lambdify((h, l), cF2Function(h, l).diff(l), [module_dictionary, "scipy"])

args = 1, 1.4

print(nd.directionaldiff(v_cF2Function, args, [1, 0], n=1))
print(f_cF2_diff_h(*args))
print()
print(nd.directionaldiff(v_cF2Function, args, [0, 1], n=1))
print(f_cF2_diff_l(*args))

12.892157132437342
12.892157132437482

5.907377428952696
5.907377428952675


#### Производительность

In [24]:
import time

In [25]:
args = 70, 80
N = 100000

start = time.time()
for _ in range(N):
    f_Z2(*args)  # без lru_cache!!!

end = time.time()

print(f"{(end - start)/N:.6e}")

6.723988e-06


In [26]:
args = 70, 80
N = 1000

start = time.time()
for _ in range(N):
    raw_Z2(*args)

end = time.time()

print(f"{(end - start)/N:.6e}")

6.912935e-04


## Моменты

### Символьные вычисления

In [27]:
m = sp.Symbol("\\left<m\\right>")
eta = sp.Symbol("\\left<\eta\\right>")

In [28]:
Z2_ferr = sp.sqrt(2)*(2*sp.pi)**2*cF2Function(h, l)/(h*sp.sqrt(l))
Z2_ferr

4*sqrt(2)*pi**2*cF2Function(h, \lambda)/(sqrt(\lambda)*h)

In [29]:
expr_m = sp.log(Z2_ferr).diff(h)/2
expr_m = expr_m.expand()
display(sp.Eq(m, expr_m))
sp.print_latex(sp.Eq(m, expr_m))

Eq(\left<m\right>, -1/(2*h) - h/(2*\lambda) + sqrt(2)*EFunction(h, \lambda, 1, 1)/(4*sqrt(\lambda)*cF2Function(h, \lambda)))

\left<m\right> = - \frac{1}{2 h} - \frac{h}{2 \lambda} + \frac{\sqrt{2} {\cal E}_{12}\left(h, \lambda\right)}{4 \sqrt{\lambda} {\cal F}^{(2)}\left(h,\lambda\right)}


In [30]:
expr_eta = sp.log(Z2_ferr).diff(l)
expr_eta = expr_eta.expand()
display(sp.Eq(eta, expr_eta))
sp.print_latex(sp.Eq(eta, expr_eta))

Eq(\left<\eta\right>, -1 - 1/(2*\lambda) + h**2/(2*\lambda**2) + sqrt(2)*EFunction(h, \lambda, 1, 0)/(2*sqrt(\lambda)*cF2Function(h, \lambda)) - sqrt(2)*h*EFunction(h, \lambda, 1, 1)/(4*\lambda**(3/2)*cF2Function(h, \lambda)))

\left<\eta\right> = -1 - \frac{1}{2 \lambda} + \frac{h^{2}}{2 \lambda^{2}} + \frac{\sqrt{2} {\cal E}_{1}\left(h, \lambda\right)}{2 \sqrt{\lambda} {\cal F}^{(2)}\left(h,\lambda\right)} - \frac{\sqrt{2} h {\cal E}_{12}\left(h, \lambda\right)}{4 \lambda^{\frac{3}{2}} {\cal F}^{(2)}\left(h,\lambda\right)}


In [31]:
E1, E2 = EFunction(h, l, 1, 1), EFunction(h, l, 1, 0)
result = sp.solve([m-expr_m, eta-expr_eta], E1, E2)
expr_E1 = result[E1].factor()
expr_E2 = result[E2].factor()

expr_cF2_1 = sp.solve(E1-expr_E1, cF2Function(h, l))[0]
expr_cF2_2 = sp.solve(E2-expr_E2, cF2Function(h, l))[0]

display(sp.Eq(cF2Function(h, l), expr_cF2_1))
sp.print_latex(sp.Eq(cF2Function(h, l), expr_cF2_1))
display(sp.Eq(cF2Function(h, l), expr_cF2_2))
sp.print_latex(sp.Eq(cF2Function(h, l), expr_cF2_2))

Eq(cF2Function(h, \lambda), sqrt(2)*sqrt(\lambda)*h*EFunction(h, \lambda, 1, 1)/(2*(2*\lambda*\left<m\right>*h + \lambda + h**2)))

{\cal F}^{(2)}\left(h,\lambda\right) = \frac{\sqrt{2} \sqrt{\lambda} h {\cal E}_{12}\left(h, \lambda\right)}{2 \left(2 \lambda \left<m\right> h + \lambda + h^{2}\right)}


Eq(cF2Function(h, \lambda), sqrt(2)*sqrt(\lambda)*EFunction(h, \lambda, 1, 0)/(2*(\lambda*\left<\eta\right> + \lambda + \left<m\right>*h + 1)))

{\cal F}^{(2)}\left(h,\lambda\right) = \frac{\sqrt{2} \sqrt{\lambda} {\cal E}_{1}\left(h, \lambda\right)}{2 \left(\lambda \left<\eta\right> + \lambda + \left<m\right> h + 1\right)}


In [32]:
subs_to_moments = {
    EFunction(h, l, 1, 1): expr_E1,
    EFunction(h, l, 1, 0): expr_E2,
    cF2Function(h, l): expr_cF2_1
}

In [33]:
expr_eta2 = Z2_ferr.diff(l, 2)/Z2_ferr
expr_eta2 = expr_eta2.expand()
expr_eta2_raw = expr_eta2
expr_eta2 = expr_eta2.subs(subs_to_moments)
expr_eta2 = expr_eta2.expand()

expr_eta2 = expr_eta2.expand().collect([EFunction(h, l, 0, 0)/EFunction(h, l, 1, 1), m, eta], sp.factor)

display(expr_eta2)
sp.print_latex(expr_eta2)

\left<\eta\right>*(-3*\lambda + h**2)/(2*\lambda**2) + \left<m\right>*h*(\lambda + 1)/\lambda**2 + (2*\lambda**2 + 1)/(2*\lambda**2) - (2*\lambda*\left<m\right>*h + \lambda + h**2)*EFunction(h, \lambda, 0, 0)/(2*\lambda**2*EFunction(h, \lambda, 1, 1))

\frac{\left<\eta\right> \left(- 3 \lambda + h^{2}\right)}{2 \lambda^{2}} + \frac{\left<m\right> h \left(\lambda + 1\right)}{\lambda^{2}} + \frac{2 \lambda^{2} + 1}{2 \lambda^{2}} - \frac{\left(2 \lambda \left<m\right> h + \lambda + h^{2}\right) {\cal E}_{}\left(h, \lambda\right)}{2 \lambda^{2} {\cal E}_{12}\left(h, \lambda\right)}


In [34]:
expr_m_upsilon = ((Z2_ferr.diff(h) - Z2_ferr.diff(h, l))/(2*Z2_ferr))/2 
expr_m_upsilon = expr_m_upsilon.expand()
expr_m_upsilon_raw = expr_m_upsilon
expr_m_upsilon = expr_m_upsilon.subs(subs_to_moments)
expr_m_upsilon = expr_m_upsilon.expand()

expr_m_upsilon = expr_m_upsilon.expand().collect([EFunction(h, l, 0, 0)/EFunction(h, l, 1, 1), m, eta], sp.factor)
display(expr_m_upsilon)
sp.print_latex(expr_m_upsilon)

\left<\eta\right>*(\lambda + h**2)/(4*\lambda*h) + \left<m\right>*(\lambda + 1)/(2*\lambda) - (2*\lambda*\left<m\right>*h + \lambda + h**2)*EFunction(h, \lambda, 0, 0)/(4*\lambda*h*EFunction(h, \lambda, 1, 1)) + 1/(4*\lambda*h)

\frac{\left<\eta\right> \left(\lambda + h^{2}\right)}{4 \lambda h} + \frac{\left<m\right> \left(\lambda + 1\right)}{2 \lambda} - \frac{\left(2 \lambda \left<m\right> h + \lambda + h^{2}\right) {\cal E}_{}\left(h, \lambda\right)}{4 \lambda h {\cal E}_{12}\left(h, \lambda\right)} + \frac{1}{4 \lambda h}


In [35]:
expr_m_par_2 = Z2_ferr.diff(h, 2)/(2*Z2_ferr) - expr_eta
expr_m_par_2 = expr_m_par_2.expand()
expr_m_par_2 = expr_m_par_2.subs(subs_to_moments).expand()

display(expr_m_par_2)
sp.print_latex(expr_m_par_2)

-2*\left<m\right>/h + 1

- \frac{2 \left<m\right>}{h} + 1


In [36]:
upsilon = sp.Symbol("\\Upsilon")
eta2 = sp.Symbol("{\\left<\\eta^2\\right>}")
m_par_2 = sp.Symbol("\\left<m_{h}^2\\right>")

dummy = sp.Dummy()
expr1 = expr_eta2.subs(EFunction(h, l, 0, 0)/EFunction(h, l, 1, 1), dummy)
expr2 = expr_m_upsilon.subs(EFunction(h, l, 0, 0)/EFunction(h, l, 1, 1), dummy)

expr_dummy = sp.solve(expr2.expand()/m-upsilon, dummy)[0]
expr1 = expr1.replace(dummy, expr_dummy).factor()
expr = sp.Eq(l*(eta2-1), sp.expand(l*(expr1-1)))

display(expr)
sp.print_latex(expr)

result = sp.solve([l*(eta2-1)-sp.expand(l*(expr1-1)), expr_m_par_2-m_par_2], h, l)
display(sp.Eq(h, result[h]))
display(sp.Eq(l, result[l]))

# display(expr_dummy.subs({h: result[h], l: result[l]}).factor())

Eq(\lambda*({\left<\eta^2\right>} - 1), 2*\Upsilon*\left<m\right>*h - 2*\left<\eta\right>)

\lambda \left({\left<\eta^2\right>} - 1\right) = 2 \Upsilon \left<m\right> h - 2 \left<\eta\right>


Eq(h, -2*\left<m\right>/(\left<m_{h}^2\right> - 1))

Eq(\lambda, 2*(-2*\Upsilon*\left<m\right>**2 - \left<\eta\right>*\left<m_{h}^2\right> + \left<\eta\right>)/(\left<m_{h}^2\right>*{\left<\eta^2\right>} - \left<m_{h}^2\right> - {\left<\eta^2\right>} + 1))

### Проверка

In [37]:
f_expr_m = sp.lambdify((h, l), expr_m, [module_dictionary_norm, "scipy"])
f_expr_eta = sp.lambdify((h, l), expr_eta, [module_dictionary_norm, "scipy"])

args = 6, 5

print(nd.directionaldiff(v_raw_Z2, args, [1, 0], n=1)/(2*raw_Z2(*args)))
print(f_expr_m(*args))
print()
print(nd.directionaldiff(v_raw_Z2, args, [0, 1], n=1)/(raw_Z2(*args)))
print(f_expr_eta(*args))

0.8840553088496662
0.8840553088496839

0.8739109152585574
0.8739109152586024


In [38]:
f_expr_cF2_1 = sp.lambdify((h, l, m, eta), expr_cF2_1, [module_dictionary, "scipy"])
f_expr_cF2_2 = sp.lambdify((h, l, m, eta), expr_cF2_2, [module_dictionary, "scipy"])

args = 1, 1.6
args_moment = f_expr_m(*args), f_expr_eta(*args)

print(f_cF2Function(*args))
print(f_expr_cF2_1(*args, *args_moment))
print(f_expr_cF2_2(*args, *args_moment))

8.34716298341246
8.34716298341246
8.34716298341246


In [39]:
f_expr_m_upsilon = sp.lambdify((h, l, m, eta), expr_m_upsilon, [module_dictionary_norm, "scipy"])
f_expr_m_upsilon_raw = sp.lambdify((h, l), expr_m_upsilon_raw, [module_dictionary_norm, "scipy"])

args = 1, 1.2
args_moment = f_expr_m(*args), f_expr_eta(*args)


def v_f_Z2_h(x): return nd.directionaldiff(v_raw_Z2, x, [1, 0], n=1)
def v_f_Z2_l(x): return nd.directionaldiff(v_raw_Z2, x, [0, 1], n=1)


# print(nd.directionaldiff(v_f_Z2_h, args, [0, 1], n=1)/(2*f_Z2(*args)))
# print(nd.directionaldiff(v_f_Z2_l, args, [1, 0], n=1)/(2*f_Z2(*args)))
print(args_moment[0]-2*f_expr_m_upsilon(*args, *args_moment))
print(args_moment[0]-2*f_expr_m_upsilon_raw(*args))

0.23230404218385675
0.2323040421838572


In [40]:
f_expr_eta2 = sp.lambdify((h, l, m, eta), expr_eta2, [module_dictionary_norm, "scipy"])
f_expr_eta2_raw = sp.lambdify((h, l), expr_eta2_raw, [module_dictionary_norm, "scipy"])

args = 3.4, 1.6
args_moment = f_expr_m(*args), f_expr_eta(*args)

print(nd.directionaldiff(v_raw_Z2, args, [0, 1], n=2)/raw_Z2(*args))
print(f_expr_eta2(*args, *args_moment))
print(f_expr_eta2_raw(*args))

0.5691720241588353
0.5691720241631888
0.5691720241631852


## Предельные случаи $\lambda \rightarrow 0$ и $h \rightarrow 0$

### Определения

In [41]:
def dawsn_to_series_oo(expr, n):
    if not isinstance(expr, F2Function):
        if expr.is_Atom:
            return expr
        return expr.__class__(*[dawsn_to_series_oo(arg, n) for arg in expr.args])

    z = expr.args[0]

    if n == 1:
        return 1/(2*z)

    return sp.factorial2(2*n-3)/(2**n*z**(2*n-1))+dawsn_to_series_oo(expr, n-1)

### $\lambda \rightarrow 0$

In [42]:
expr = Z2_ferr.rewrite(sp.exp).rewrite(sp.erfi).factor()
expr = sp.limit(expr, l, 0).expand()
display(expr)

expr = expr.rewrite(sp.sin).expand().factor()
expr

4*pi**2*exp(2*h)/h**2 - 8*pi**2/h**2 + 4*pi**2*exp(-2*h)/h**2

8*pi**2*(cosh(2*h) - 1)/h**2

In [43]:
expr = expr_m.rewrite(sp.exp).rewrite(sp.erfi)
expr = sp.limit(expr, l, 0).factor()
display(expr)

_exp_m = sp.solve(expr-m, sp.exp(2*h))[0]

expr = expr.expand().rewrite(sp.tanh).factor().expand()
expr

(h*exp(2*h) + h - exp(2*h) + 1)/(h*(exp(h) - 1)*(exp(h) + 1))

1/tanh(h) - 1/h

In [44]:
expr = expr_eta.rewrite(sp.exp)
expr = dawsn_to_series_oo(expr, 3).factor()
expr = sp.limit(expr, l, 0).factor()
display(expr)
sp.print_latex(expr)

expr = expr.expand().subs(sp.exp(2*h), _exp_m).factor()
expr

(h*exp(2*h) + h - exp(2*h) + 1)**2/(h**2*(exp(h) - 1)**2*(exp(h) + 1)**2)

\frac{\left(h e^{2 h} + h - e^{2 h} + 1\right)^{2}}{h^{2} \left(e^{h} - 1\right)^{2} \left(e^{h} + 1\right)^{2}}


\left<m\right>**2

In [45]:
expr = (expr_m_upsilon/m).subs({m: expr_m, eta: expr_eta}).rewrite(sp.exp)
expr = dawsn_to_series_oo(expr, 4).factor()
expr = sp.limit(expr, l, 0).factor()
display(expr)
sp.print_latex(expr)

expr = expr.expand().subs(sp.exp(2*h), _exp_m).factor().expand()
display(expr)
sp.print_latex(expr)

(h*exp(2*h) + h - exp(2*h) + 1)/(h**2*(exp(h) - 1)*(exp(h) + 1))

\frac{h e^{2 h} + h - e^{2 h} + 1}{h^{2} \left(e^{h} - 1\right) \left(e^{h} + 1\right)}


\left<m\right>/h

\frac{\left<m\right>}{h}


In [46]:
expr = (expr_eta2-1).subs({m: expr_m, eta: expr_eta}).rewrite(sp.exp).factor()
expr = dawsn_to_series_oo(expr, 5).factor()
expr = sp.limit(expr, l, 0).factor()
display(expr)
sp.print_latex(expr)

expr = expr.expand().subs(sp.exp(2*h), _exp_m).factor().expand()
display(expr)
sp.print_latex(expr)

-2*(h*exp(2*h) + h - exp(2*h) + 1)*(2*h**2*exp(2*h) - 2*h**2 - 3*h*exp(2*h) - 3*h + 3*exp(2*h) - 3)/(h**4*(exp(h) - 1)**2*(exp(h) + 1)**2)

- \frac{2 \left(h e^{2 h} + h - e^{2 h} + 1\right) \left(2 h^{2} e^{2 h} - 2 h^{2} - 3 h e^{2 h} - 3 h + 3 e^{2 h} - 3\right)}{h^{4} \left(e^{h} - 1\right)^{2} \left(e^{h} + 1\right)^{2}}


6*\left<m\right>**2/h**2 - 4*\left<m\right>/h

\frac{6 \left<m\right>^{2}}{h^{2}} - \frac{4 \left<m\right>}{h}


### $h \rightarrow 0$

In [47]:
expr = Z2_ferr.rewrite(sp.exp).rewrite(sp.erfi)
expr = sp.limit(expr, h, 0).expand()
display(expr)
sp.print_latex(expr)

expr = expr.rewrite(sp.sin).factor()
display(expr)
sp.print_latex(expr)

8*pi**2*exp(\lambda)/\lambda - 8*pi**2*exp(-\lambda)/\lambda

\frac{8 \pi^{2} e^{\lambda}}{\lambda} - \frac{8 \pi^{2} e^{- \lambda}}{\lambda}


16*pi**2*sinh(\lambda)/\lambda

\frac{16 \pi^{2} \sinh{\left(\lambda \right)}}{\lambda}


In [48]:
expr = expr_m.rewrite(sp.exp).rewrite(sp.erfi)
expr = sp.limit(expr, h, 0).expand()
display(expr)

0

In [49]:
expr = expr_eta.rewrite(sp.exp).rewrite(sp.erfi)
expr = sp.limit(expr, h, 0).factor()
display(expr)
sp.print_latex(expr)

expr = expr.expand().rewrite(sp.tanh).factor().expand()
display(expr)
sp.print_latex(expr)

(\lambda*exp(2*\lambda) + \lambda - exp(2*\lambda) + 1)/(\lambda*(exp(\lambda) - 1)*(exp(\lambda) + 1))

\frac{\lambda e^{2 \lambda} + \lambda - e^{2 \lambda} + 1}{\lambda \left(e^{\lambda} - 1\right) \left(e^{\lambda} + 1\right)}


1/tanh(\lambda) - 1/\lambda

\frac{1}{\tanh{\left(\lambda \right)}} - \frac{1}{\lambda}


In [50]:
expr = (expr_m_upsilon/m).subs({m: expr_m, eta: expr_eta}).rewrite(sp.exp).rewrite(sp.erfi)
expr = sp.limit(expr.factor(), h, 0).factor()
display(expr)
sp.print_latex(expr)

expr = expr.rewrite(sp.tanh).factor()
expr = expr.subs(sp.tanh(l), 1/(eta+1/l)).factor()
display(expr)
sp.print_latex(expr)

(\lambda*exp(2*\lambda) + \lambda - exp(2*\lambda) + 1)/(\lambda*(2*\lambda*exp(2*\lambda) - exp(2*\lambda) + 1))

\frac{\lambda e^{2 \lambda} + \lambda - e^{2 \lambda} + 1}{\lambda \left(2 \lambda e^{2 \lambda} - e^{2 \lambda} + 1\right)}


\left<\eta\right>/(\lambda*(\left<\eta\right> + 1))

\frac{\left<\eta\right>}{\lambda \left(\left<\eta\right> + 1\right)}


In [51]:
expr = expr_eta2.subs({m: expr_m, eta: expr_eta}).rewrite(sp.exp).rewrite(sp.erfi)
expr = sp.limit(expr.factor(), h, 0).factor()
display(expr)
sp.print_latex(expr)

expr = expr.expand().rewrite(sp.tanh).factor().expand()
expr = expr.subs(1/sp.tanh(l), eta+1/l).expand()
display(expr)
sp.print_latex(expr)

(\lambda**2*exp(2*\lambda) - \lambda**2 - 2*\lambda*exp(2*\lambda) - 2*\lambda + 2*exp(2*\lambda) - 2)/(\lambda**2*(exp(\lambda) - 1)*(exp(\lambda) + 1))

\frac{\lambda^{2} e^{2 \lambda} - \lambda^{2} - 2 \lambda e^{2 \lambda} - 2 \lambda + 2 e^{2 \lambda} - 2}{\lambda^{2} \left(e^{\lambda} - 1\right) \left(e^{\lambda} + 1\right)}


1 - 2*\left<\eta\right>/\lambda

1 - \frac{2 \left<\eta\right>}{\lambda}


## Генерация кода C++ для реализации метода Ньютона

In [52]:
expr_G1 = expr_m-m
expr_G2 = expr_eta-eta

expr_G1_diff_h = expr_G1.diff(h).expand()
expr_G1_diff_l = expr_G1.diff(l).expand()

expr_G2_diff_h = expr_G2.diff(h).expand()
expr_G2_diff_l = expr_G2.diff(l).expand()

display(expr_G2_diff_h)
display(expr_G2_diff_l)

-EFunction(h, \lambda, 1, 0)*EFunction(h, \lambda, 1, 1)/(2*\lambda*cF2Function(h, \lambda)**2) + h*EFunction(h, \lambda, 1, 1)**2/(4*\lambda**2*cF2Function(h, \lambda)**2) + h/\lambda**2 + sqrt(2)*EFunction(h, \lambda, 0, 0)/(2*sqrt(\lambda)*cF2Function(h, \lambda)) + sqrt(2)*EFunction(h, \lambda, 1, 1)/(2*sqrt(\lambda)*cF2Function(h, \lambda)) - sqrt(2)*EFunction(h, \lambda, 1, 1)/(4*\lambda**(3/2)*cF2Function(h, \lambda)) - sqrt(2)*h**2*EFunction(h, \lambda, 1, 1)/(4*\lambda**(5/2)*cF2Function(h, \lambda))

-EFunction(h, \lambda, 1, 0)**2/(2*\lambda*cF2Function(h, \lambda)**2) + h*EFunction(h, \lambda, 1, 0)*EFunction(h, \lambda, 1, 1)/(2*\lambda**2*cF2Function(h, \lambda)**2) + 1/(2*\lambda**2) - h**2*EFunction(h, \lambda, 1, 1)**2/(8*\lambda**3*cF2Function(h, \lambda)**2) - h**2/\lambda**3 + sqrt(2)*EFunction(h, \lambda, 1, 0)/(sqrt(\lambda)*cF2Function(h, \lambda)) - sqrt(2)*h*EFunction(h, \lambda, 0, 0)/(4*\lambda**(3/2)*cF2Function(h, \lambda)) - sqrt(2)*h*EFunction(h, \lambda, 1, 1)/(4*\lambda**(3/2)*cF2Function(h, \lambda)) - sqrt(2)*EFunction(h, \lambda, 1, 0)/(4*\lambda**(3/2)*cF2Function(h, \lambda)) - sqrt(2)*h**2*EFunction(h, \lambda, 1, 0)/(4*\lambda**(5/2)*cF2Function(h, \lambda)) + 3*sqrt(2)*h*EFunction(h, \lambda, 1, 1)/(8*\lambda**(5/2)*cF2Function(h, \lambda)) + sqrt(2)*h**3*EFunction(h, \lambda, 1, 1)/(8*\lambda**(7/2)*cF2Function(h, \lambda))

In [53]:
from sympy.printing.c import C99CodePrinter


class CPrinter(C99CodePrinter):
    def _print_Function(self, expr):
        if isinstance(expr, F2Function):
            return f"gsl_sf_dawson({self._print(expr.args[0])})"

        return super()._print_Function(expr)

    def _print_Rational(self, expr):
        if expr.equals(sp.Rational(1, 2)):
            return ".5"
        if expr.equals(sp.Rational(1, 4)):
            return ".25"
        if expr.equals(sp.Rational(3, 2)):
            return "1.5"
        if expr.equals(sp.Rational(1, 8)):
            return "0.125"
        if expr.equals(sp.Rational(3, 8)):
            return "0.375"
        return super()._print_Rational(expr)

    def _print_Pow(self, expr):
        arg, power = expr.args
        if power.is_integer:
            return "("+"*".join([self._print(arg)]*int(power.evalf()))+")"

        return super()._print_Pow(expr)


def get_ccode(expr):
    return CPrinter().doprint(expr)

In [54]:
subs = {
    EFunction(h, l, 1, 1): sp.Symbol("E12_cF2")*cF2Function(h, l),
    EFunction(h, l, 1, 0): sp.Symbol("E1_cF2")*cF2Function(h, l),
    EFunction(h, l, 0, 0): sp.Symbol("E_cF2")*cF2Function(h, l),
    m: sp.Symbol("m"), eta: sp.Symbol("eta"),
    sp.exp(-2*h): sp.Symbol("_exp_2h"),
    sp.exp(-2*l): sp.Symbol("_exp_2l"),
    l: sp.Symbol("l")
}


def _optimize_pow(expr, symbol, name_symbol):
    result = 0
    for key, value in sp.collect(expr, symbol, evaluate=False).items():
        if isinstance(key, sp.Pow):
            power = key.args[1]
            if power.is_integer:
                if power < 0:
                    result += sp.Symbol(f"_{name_symbol}")**(-power)*value
                    continue
                result += symbol**power*value
                continue

            if (2*power).is_integer:
                if power < 0:
                    power = int(power + 1/2)
                    result += sp.Symbol(f"_sqrt_{name_symbol}") * \
                        sp.Symbol(f"_{name_symbol}")**(-power)*value
                    continue
                if power > 0:
                    power = int(power - 1/2)
                    result += sp.Symbol(f"_sqrt_{name_symbol}") * \
                        symbol**(power)*value
                    continue

        result += key*value

    return result


def optimize(expr):
    expr = _optimize_pow(expr, l, "l")
    expr = _optimize_pow(expr, h, "h")
    expr = expr.subs(subs)
    return expr

In [55]:
print("double _l = 1/l;")
print("double _h = 1/h;")
print("double _sqrt_l = sqrt(_l);")
print("double _exp_2l = exp(-2*l);")
print("double _exp_2h = exp(-2*h);")
val_cF2 = cF2Function(h, l).rewrite(sp.exp)
val_cF2 = val_cF2.replace(F2Function, lambda *args: F2Function(optimize(args[0]))*sp.exp(-2*h-l))
print("double cF2 = ", get_ccode(optimize(val_cF2))+";")
print("double _cF2 = 1/cF2;")
print("double E12_cF2 = _cF2*(", get_ccode(optimize(sp.expand(EFunction(h, l, 1, 1).rewrite(sp.exp)*sp.exp(-2*h-l))))+");")
print("double E1_cF2 = _cF2*(", get_ccode(optimize(sp.expand(EFunction(h, l, 1, 0).rewrite(sp.exp)*sp.exp(-2*h-l))))+");")
print("double E_cF2 = _cF2*(", get_ccode(optimize(sp.expand(EFunction(h, l, 0, 0).rewrite(sp.exp)*sp.exp(-2*h-l))))+");")
print()
print("double G1 =", get_ccode(optimize(expr_G1))+";")
print("double G1_diff_h =", get_ccode(optimize(expr_G1_diff_h))+";")
print("double G1_diff_l =", get_ccode(optimize(expr_G1_diff_l))+";")
print()
print("double G2 =", get_ccode(optimize(expr_G2))+";")
print("double G2_diff_h =", get_ccode(optimize(expr_G2_diff_h))+";")
print("double G2_diff_l =", get_ccode(optimize(expr_G2_diff_l))+";")

double _l = 1/l;
double _h = 1/h;
double _sqrt_l = sqrt(_l);
double _exp_2l = exp(-2*l);
double _exp_2h = exp(-2*h);
double cF2 =  (_exp_2h*_exp_2h)*gsl_sf_dawson((.5)*M_SQRT2*_sqrt_l*(h - 2*l)) - 2*_exp_2h*_exp_2l*gsl_sf_dawson((.5)*M_SQRT2*_sqrt_l*h) + gsl_sf_dawson((.5)*M_SQRT2*_sqrt_l*(h + 2*l));
double _cF2 = 1/cF2;
double E12_cF2 = _cF2*( (_exp_2h*_exp_2h) - 2*_exp_2h*_exp_2l + 1);
double E1_cF2 = _cF2*( 1 - (_exp_2h*_exp_2h));
double E_cF2 = _cF2*( (_exp_2h*_exp_2h) + 2*_exp_2h*_exp_2l + 1);

double G1 = (.25)*M_SQRT2*E12_cF2*_sqrt_l - .5*_h - .5*_l*h - m;
double G1_diff_h = (.25)*M_SQRT2*E12_cF2*_l*_sqrt_l*h + (.5)*M_SQRT2*E1_cF2*_sqrt_l + (.5)*(_h*_h) + _l*(-.25*(E12_cF2*E12_cF2) - 1.0/2.0);
double G1_diff_l = -.25*E12_cF2*E1_cF2*_l - 0.125*M_SQRT2*E12_cF2*(_l*_l)*_sqrt_l*(h*h) - 0.125*M_SQRT2*E12_cF2*_l*_sqrt_l + (_l*_l)*((0.125)*(E12_cF2*E12_cF2)*h + (.5)*h) + _sqrt_l*((.25)*M_SQRT2*E12_cF2 + (.25)*M_SQRT2*E_cF2);

double G2 = -.25*M_SQRT2*E12_cF2*_l*_sqrt_l*h + (.5)*M_SQRT2

# Ферромагнетик с $\left<m_1\right> \neq \left<m_2\right>$

## Выражение для $Z^{(2)}$

### Аргументы

In [56]:
h1, h2, l = sp.symbols("h_1 h_2 \\lambda", positive=True)
s1, s2 = SigmaSymbol("\\sigma_1"), SigmaSymbol("\\sigma_2")
t = sp.Symbol("t")

In [57]:
arg_exp = t**2/(2*l)*h1/h2 + s1*t
bound = h2+s2*l

arg_exp = arg_exp.replace(t, t-s1*l*h2/h1).expand()
bound += s1*l*h2/h1

arg_exp = arg_exp.replace(t, t*sp.sqrt(2*l*h2/h1))
bound /= sp.sqrt(2*l*h2/h1)

display(arg_exp)
display(bound)

-\lambda*h_2/(2*h_1) + t**2

sqrt(2)*sqrt(h_1)*(\sigma_1*\lambda*h_2/h_1 + \sigma_2*\lambda + h_2)/(2*sqrt(\lambda)*sqrt(h_2))

In [58]:
expr = bound**2 + (arg_exp - t**2)
expr = expr.expand()
expr -= (h2**2+l**2)/(2*l*h2/h1)
expr = expr.expand()

display(expr)

\sigma_1*\sigma_2*\lambda + \sigma_1*h_2 + \sigma_2*h_1

### Производные $\cal{F}^{(2)}$

In [59]:
_sub_expr_E = sp.exp(h1*s1+h2*s2+l*s1*s2).expand()
_sub_expr_cF2 = F2Function((h2+s1*l+s2*l*h2/h1)/sp.sqrt(2*l*h2/h1)).expand()

_dummy_subs = {
    _sub_expr_E*_sub_expr_cF2: sp.Dummy(),
    _sub_expr_E: sp.Dummy()
}


def sigmas_sum(expr):
    expr = expr.expand().subs(_dummy_subs)

    terms = sp.collect(expr, _dummy_subs.values(), evaluate=False)
    for dummy in _dummy_subs.values():
        if dummy in terms:
            terms[dummy] = sp.Poly(terms[dummy], [s1, s2]).as_dict()
    for key, dummy in _dummy_subs.items():
        if dummy in terms:
            terms[key] = terms.pop(dummy)

    return terms

In [60]:
class Z2nsSigmasFunctionBase(sp.Function):
    def __init__(self, name, *args, **kwargs):
        self.name = name

    def _latex(self, printer, exp=None):
        args = list(map(printer.doprint, self.args[:3]))
        args = f"\\left({args[0]}, {args[1]}, {args[2]}\\right)"

        indexes = ""
        for i, item in enumerate(self.args[3:]):
            if item:
                indexes += str(i+1)
        name = f"{self.name}"+"_{" + indexes + "}"

        if exp is None:
            return name+args
        else:
            exp = printer.doprint(exp)
            return f"{name}^{exp}"+args

    def get_sigmas_factor(self):
        factor = 1
        if self.args[3]:
            factor *= s1
        if self.args[4]:
            factor *= s2
        return factor

In [61]:
class EFunction_ns(Z2nsSigmasFunctionBase):
    def __init__(self, *args, **kwargs):
        super().__init__("{\\cal E}", *args, **kwargs)

    def fdiff(self, argindex=1):
        factor = self.get_sigmas_factor()
        expr = (factor*_sub_expr_E).diff([h1, h2, l][argindex-1])
        terms = sigmas_sum(expr)

        subs = dict(zip([h1, h2, l], self.args[:3]))

        result = 0
        for key, item in terms[_sub_expr_E].items():
            result += item.subs(subs)*EFunction_ns(*self.args[:3], *key)

        return result
        
    def _eval_rewrite(self, rule, args, **hints):
        h1, h2, l, arg_s1, arg_s2 = args
        if rule == sp.exp:
            result = 0
            for s1 in SIGMAS:
                for s2 in SIGMAS:
                    term = sp.exp(s1*h1+s2*h2+s1*s2*l)
                    if arg_s1:
                        term *= s1
                    if arg_s2:
                        term *= s2
                    result += term
            return result
    
    @classmethod
    def eval(cls, *args):
        if args[0] == args[1]:
            return EFunction(args[0], *args[2:])

In [62]:
EFunction_ns(h1, h2, l, 1, 1).diff(h1)

EFunction_ns(h_1, h_2, \lambda, 0, 1)

In [63]:
class cF2Function_ns(sp.Function):
    def _latex(self, printer, exp=None):
        args = list(map(printer.doprint, self.args[:3]))
        args = f"\\left({args[0]}, {args[1]}, {args[2]}\\right)"
        if exp is None:
            return "{\\cal F}^{(2)}"+args
        else:
            exp = printer.doprint(exp)
            return f"\\left({self._latex(printer)}\\right)^{exp}"

    def fdiff(self, argindex=1):
        expr = (s1*s2*_sub_expr_E*_sub_expr_cF2).diff([h1, h2, l][argindex-1])
        terms = sigmas_sum(expr)

        subs = dict(zip([h1, h2, l], self.args[:3]))

        result = 0
        for key, item in terms[_sub_expr_E].items():
            result += item.subs(subs)*EFunction_ns(*self.args, *key)
        
        _term = terms[_sub_expr_E*_sub_expr_cF2][(1, 1)].factor().subs(subs)
        result += _term*cF2Function_ns(*self.args)
        return result

    def _eval_rewrite(self, rule, args, **hints):
        h1, h2, l = args
        if rule == sp.exp:
            result = 0
            for s1 in SIGMAS:
                for s2 in SIGMAS:
                    term = sp.exp(s1*h1+s2*h2+s1*s2*l)
                    term *= F2Function((h2+s1*l+s2*l*h2/h1)/sp.sqrt(2*l*h2/h1))
                    term *= s1*s2
                    result += term
            return result

        return result

    @classmethod
    def eval(cls, *args):
        if args[0] == args[1]:
            return cF2Function(args[0], args[2])

In [64]:
expr = cF2Function_ns(h1, h2, l).diff(l).expand()

display(expr)
sp.print_latex(expr)

-h_1*cF2Function_ns(h_1, h_2, \lambda)/(2*h_2) - h_2*cF2Function_ns(h_1, h_2, \lambda)/(2*h_1) + h_1*h_2*cF2Function_ns(h_1, h_2, \lambda)/(2*\lambda**2) + sqrt(2)*sqrt(h_1)*EFunction_ns(h_1, h_2, \lambda, 0, 1)/(4*sqrt(\lambda)*sqrt(h_2)) + sqrt(2)*sqrt(h_2)*EFunction_ns(h_1, h_2, \lambda, 1, 0)/(4*sqrt(\lambda)*sqrt(h_1)) - sqrt(2)*sqrt(h_1)*sqrt(h_2)*EFunction_ns(h_1, h_2, \lambda, 1, 1)/(4*\lambda**(3/2))

- \frac{h_{1} {\cal F}^{(2)}\left(h_{1}, h_{2}, \lambda\right)}{2 h_{2}} - \frac{h_{2} {\cal F}^{(2)}\left(h_{1}, h_{2}, \lambda\right)}{2 h_{1}} + \frac{h_{1} h_{2} {\cal F}^{(2)}\left(h_{1}, h_{2}, \lambda\right)}{2 \lambda^{2}} + \frac{\sqrt{2} \sqrt{h_{1}} {\cal E}_{2}\left(h_{1}, h_{2}, \lambda\right)}{4 \sqrt{\lambda} \sqrt{h_{2}}} + \frac{\sqrt{2} \sqrt{h_{2}} {\cal E}_{1}\left(h_{1}, h_{2}, \lambda\right)}{4 \sqrt{\lambda} \sqrt{h_{1}}} - \frac{\sqrt{2} \sqrt{h_{1}} \sqrt{h_{2}} {\cal E}_{12}\left(h_{1}, h_{2}, \lambda\right)}{4 \lambda^{\frac{3}{2}}}


### Численный расчет

#### Определения

In [65]:
@lru_cache(maxsize=1)
def _Z2_ns_E_terms(h1, h2, l):
    def func(s1, s2): return np.exp(s1*h1+s2*h2+s1*s2*l)
    return np.asarray([[func(s1, s2) for s2 in SIGMAS] for s1 in SIGMAS])

@lru_cache(maxsize=1)
def _Z2_ns_F_terms(h1, h2, l):
    def func(s1, s2): return f_F2Function((h2+s1*l+s2*l*h2/h1)/np.sqrt(2*l*h2/h1))
    return np.asarray([[func(s1, s2) for s2 in SIGMAS] for s1 in SIGMAS])

In [66]:
def f_cF2Function_ns(h1, h2, l):
    _terms = _Z2_ns_E_terms(h1, h2, l)*_Z2_ns_F_terms(h1, h2, l)
    def _sigmas_func_12(s1, s2): return s1*s2
    return _Z2_sum_terms(_terms, _sigmas_func_12)

def v_cF2Function_ns(x): return f_cF2Function_ns(*x)

In [67]:
def f_Z2_ns(h1, h2, l):
    return np.sqrt(2)*(2*np.pi)**2/(np.sqrt(h1*h2*l))*f_cF2Function_ns(h1, h2, l)

In [71]:
def raw_Z2_ns(h1, h2, l):
    def _sqrt(x): return np.sqrt(h2**2+l**2+2*h2*l*x)
    def func(x): return np.exp(h1*x)*f_Z1(_sqrt(x))
    return 2*np.pi*quad(func, -1, 1)[0]

def v_raw_Z2_ns(x): return raw_Z2_ns(*x)

In [72]:
def f_EFunction_ns(h1, h2, l, arg_s1, arg_s2):
    def _sigmas_func(s1, s2):
        result = 1
        if arg_s1:
            result *= s1
        if arg_s2:
            result *= s2
        return result
    return _Z2_sum_terms(_Z2_ns_E_terms(h1, h2, l), _sigmas_func)

module_ns_dictionary = {"cF2Function_ns": f_cF2Function_ns, "EFunction_ns": f_EFunction_ns, "F2Function": f_F2Function}

#### Проверка

In [73]:
args = 3, 2, 1.5
print(raw_Z2_ns(*args))
print(f_Z2_ns(*args))

2056.4127882868374
2056.4127882868374


In [74]:
f_cF2_ns_diff_h1 = sp.lambdify((h1, h2, l), cF2Function_ns(h1, h2, l).diff(h1), [module_ns_dictionary, "scipy"])
f_cF2_ns_diff_h2 = sp.lambdify((h1, h2, l), cF2Function_ns(h1, h2, l).diff(h2), [module_ns_dictionary, "scipy"])
f_cF2_ns_diff_l  = sp.lambdify((h1, h2, l), cF2Function_ns(h1, h2, l).diff(l),  [module_ns_dictionary, "scipy"])

args = 1, 2, 1.4

print(nd.directionaldiff(v_cF2Function_ns, args, [1, 0, 0], n=1))
print(f_cF2_ns_diff_h1(*args))
print()
print(nd.directionaldiff(v_cF2Function_ns, args, [0, 1, 0], n=1))
print(f_cF2_ns_diff_h2(*args))
print()
print(nd.directionaldiff(v_cF2Function_ns, args, [0, 0, 1], n=1))
print(f_cF2_ns_diff_l(*args))

16.26477756622653
16.26477756622638

14.133140858579903
14.133140858580846

14.618960735571534
14.618960735572816


## Моменты

In [75]:
Z2_ns = sp.sqrt(2)*(2*sp.pi)**2/sp.sqrt(h1*h2*l)*cF2Function_ns(h1, h2, l)
Z2_ns

4*sqrt(2)*pi**2*cF2Function_ns(h_1, h_2, \lambda)/(sqrt(\lambda)*sqrt(h_1)*sqrt(h_2))

### $\Psi$ в симметричном случае

#### Численный расчет

In [78]:
from scipy.special import iv

def _raw_Z2_general(h1_per_2, h1_par_2, h2, l):
    def _sqrt(x): return np.sqrt(h2**2+l**2+2*h2*l*np.cos(x))
    def func(x): return iv(0, h1_per_2*np.sin(x))*np.exp(h1_par_2*np.cos(x))*f_Z1(_sqrt(x))*np.sin(x)
    return 2*np.pi*quad(func, 0, np.pi)[0]

def raw_Z2_general(h1, h2, l):
    h2_norm = np.linalg.norm(h2)
    h1_par_2 = h1.dot(h2)/h2_norm
    h1_per_2 = np.sqrt(np.abs(h1.dot(h1) - h1_par_2**2))
    return _raw_Z2_general(h1_per_2, h1_par_2, h2_norm, l)

In [79]:
arg_h1 = np.asarray([0, 0, 2])
arg_h2 = np.asarray([0, 0, 1])
arg_l = 2

print(raw_Z2_general(arg_h1, arg_h2, arg_l))
print(raw_Z2_ns(2, 1, 2))

782.8469849077993
782.8469849077994


In [80]:
def calc_diff(func, args):
    if len(args) == 0:
        return func
    func = calc_diff(func, args[1:])
    def diff(x0):
        x1 = np.copy(x0)
        x2 = np.copy(x0)
        x1[args[0]] += 1e-4
        x2[args[0]] -= 1e-4
        return (func(x1)-func(x2))/(2*1e-4)
    return diff

In [81]:
print(calc_diff(lambda x: np.sin(x), [0, 0])([1.]))
print(np.sin(1))

[-0.84147098]
0.8414709848078965


In [82]:
def raw_psi(h, l, nk):
    def func(args):
        h, l, gamma1, gamma2  = args
        h = np.asarray([0, 0, h])
        h1 = h + gamma1*nk
        h2 = h + gamma2*nk
        return raw_Z2_general(h1, h2, l)
    return (calc_diff(func, [1, 3, 3])([h, l, 0, 0]) - calc_diff(func, [2, 3])([h, l, 0, 0]))/raw_Z2(h, l)
    # return (calc_diff(func, [3, 3])([h, l, 0, 0]))/raw_Z2(h, l)

In [83]:
args = 1.4, 1.6
vnk = np.asarray([0, 0, 1])

print(raw_psi(*args, vnk))
print((1-f_expr_eta(*args))*f_expr_m(*args)**3*(0.46134-1.3836))

-0.06225928895713271
-0.06146556308531417


#### Символьные вычисления

In [84]:
from sympy.vector import CoordSys3D, Del

N = CoordSys3D("N")
delop = Del()

vh1, vh2, vh3 = sp.symbols("h_1 h_2 h_3")
vh = vh1*N.i + vh2*N.j + vh3*N.k

nk1, nk2, nk3 = sp.symbols("n_{1K} n_{2K} n_{3K}")
nk = nk1*N.i + nk2*N.j + nk3*N.k

In [85]:
gamma1, gamma2 = sp.symbols("\\gamma_1 \\gamma_2", positive=True)

In [86]:
v1 = vh + gamma1*nk
v2 = vh + gamma2*nk

v1_par_2 = v1.dot(v2)/v2.magnitude()
v1_per_2 = sp.sqrt(v1.dot(v1) - v1_par_2**2)
v2_magnitude = v2.magnitude()

In [87]:
def _diff_func(func, args): 
    if len(args) == 0:
        return func

    arg = args[0]
    if isinstance(func, sp.Function):
        func = func.fdiff(arg)
        return _diff_func(func, args[1:])
    if isinstance(func, sp.Subs):
        return sp.Subs(_diff_func(func.args[0], args), *func.args[1:])
    if isinstance(func, sp.Derivative):
        return sp.Derivative(_diff_func(func.args[0], args), *func.args[1:])

In [88]:
Z = sp.Function("Z")(v1_per_2, v1_par_2, v2_magnitude, l)

def two_diff_func(func, arg1, arg2):
    args = func.args
    result = 0
    for i, arg in enumerate(args):
        result += arg.diff(arg1, arg2).subs({gamma2: 0}).factor()*_diff_func(func, [i+1])

        factor = arg.diff(arg1).subs({gamma2: 0})
        for j, j_arg in enumerate(args):
            term = (factor*j_arg.diff(arg2)).subs({gamma2: 0}).factor()
            result += term*_diff_func(func, [i+1, j+1])
    return result

expr_psi = (two_diff_func(Z, gamma1, gamma1).diff(l) - two_diff_func(Z, gamma1, gamma2)).subs({gamma1: 0, gamma2: 0})
expr_psi = expr_psi.subs(vh.dot(vh), h**2).expand()

def v_per_diff_eval(*args):
    func = args[0]
    diff_args = args[1:]
    per_arg = func.args[0]
    non_per_diff_args = tuple(filter(lambda item: item[0] != per_arg, diff_args))
    per_diff_args = tuple(filter(lambda item: item[0] == per_arg, diff_args))

    if len(per_diff_args) == 0:
        return sp.Derivative(*args)
    
    _, n = per_diff_args[0]

    if n == 2:
        func = func - _diff_func(func, (2, 2))
        func /= 2
    elif n == 1:
        return 0
    else:
        return sp.Derivative(*args)

    if len(non_per_diff_args) > 0:
        func = sp.Derivative(func, *non_per_diff_args)

    return func

expr_psi = expr_psi.replace(sp.Derivative, v_per_diff_eval).doit()

expr_psi = expr_psi.replace(sp.Function("Z"), lambda *args: Z2_ns.subs(dict(zip([h1, h2, l], args[1:])))).doit()
expr_psi = (expr_psi/Z2_ns.subs({h1: h, h2: h})).expand()
expr_psi = expr_psi.subs({E1: expr_E1, E2: expr_E2})
expr_psi = expr_psi.replace(cF2Function(h, l), expr_cF2_1)
expr_psi = expr_psi.factor()
expr_psi

-(2*h_1**2*n_{1K}**2 - h_1**2*n_{2K}**2 - h_1**2*n_{3K}**2 + 6*h_1*h_2*n_{1K}*n_{2K} + 6*h_1*h_3*n_{1K}*n_{3K} - h_2**2*n_{1K}**2 + 2*h_2**2*n_{2K}**2 - h_2**2*n_{3K}**2 + 6*h_2*h_3*n_{2K}*n_{3K} - h_3**2*n_{1K}**2 - h_3**2*n_{2K}**2 + 2*h_3**2*n_{3K}**2)*(-3*\lambda**2*\left<\eta\right>*EFunction(h, \lambda, 1, 1) + 6*\lambda**2*\left<m\right>*h*EFunction(h, \lambda, 0, 0) - 6*\lambda**2*\left<m\right>*h*EFunction(h, \lambda, 1, 1) + 3*\lambda**2*EFunction(h, \lambda, 0, 0) + 2*\lambda*\left<m\right>*h**3*EFunction(h, \lambda, 0, 0) - 2*\lambda*\left<m\right>*h**3*EFunction(h, \lambda, 1, 1) - 6*\lambda*\left<m\right>*h*EFunction(h, \lambda, 1, 1) + 4*\lambda*h**2*EFunction(h, \lambda, 0, 0) - 3*\lambda*EFunction(h, \lambda, 1, 1) - \left<\eta\right>*h**4*EFunction(h, \lambda, 1, 1) - 2*\left<m\right>*h**3*EFunction(h, \lambda, 1, 1) + h**4*EFunction(h, \lambda, 0, 0) - h**2*EFunction(h, \lambda, 1, 1))/(8*\lambda**2*h**4*EFunction(h, \lambda, 1, 1))

In [89]:
f_expr_psi = sp.lambdify((vh1, vh2, vh3, nk1, nk2, nk3, h, l, m, eta), expr_psi, module_dictionary)
 
args = 1.4, 1.6
vh_args =  np.asarray([0, 0, args[0]])
nk_args = np.asarray([0, 1, 1.])
nk_args /= np.linalg.norm(nk_args)
args_moment = f_expr_m(*args), f_expr_eta(*args)

print(f_expr_psi(*vh_args, *nk_args, *args, *args_moment))
print(raw_psi(*args, nk_args))
print((1-f_expr_eta(*args))*f_expr_m(*args)**3*(0.46134-1.3836*vh_args.dot(nk_args)**2))

-0.015562017813131329
-0.01560789008725465
-0.059621316276716976


In [90]:
dummy = sp.Dummy()
_expr = expr_m_upsilon.subs(EFunction(h, l, 0, 0)/EFunction(h, l, 1, 1), dummy)
expr_psi_upsilon = expr_psi.expand().subs(EFunction(h, l, 0, 0)/EFunction(h, l, 1, 1), dummy)

expr_dummy = sp.solve(_expr.expand()/m-upsilon, dummy)[0]
expr_psi_upsilon = expr_psi_upsilon.replace(dummy, expr_dummy).factor().expand()
expr_psi_upsilon.factor()

(3*\Upsilon*\lambda*\left<m\right> + \Upsilon*\left<m\right>*h**2 - \left<\eta\right>*h)*(2*h_1**2*n_{1K}**2 - h_1**2*n_{2K}**2 - h_1**2*n_{3K}**2 + 6*h_1*h_2*n_{1K}*n_{2K} + 6*h_1*h_3*n_{1K}*n_{3K} - h_2**2*n_{1K}**2 + 2*h_2**2*n_{2K}**2 - h_2**2*n_{3K}**2 + 6*h_2*h_3*n_{2K}*n_{3K} - h_3**2*n_{1K}**2 - h_3**2*n_{2K}**2 + 2*h_3**2*n_{3K}**2)/(2*\lambda*h**3)

In [91]:
expr_psi_upsilon_Q0 = expr_psi_upsilon.replace(l, 2/(eta2-1)*(upsilon*m*h-eta)).factor()
expr_psi_upsilon_Q0

(6*\Upsilon*\left<m\right> + h*{\left<\eta^2\right>} - h)*(2*h_1**2*n_{1K}**2 - h_1**2*n_{2K}**2 - h_1**2*n_{3K}**2 + 6*h_1*h_2*n_{1K}*n_{2K} + 6*h_1*h_3*n_{1K}*n_{3K} - h_2**2*n_{1K}**2 + 2*h_2**2*n_{2K}**2 - h_2**2*n_{3K}**2 + 6*h_2*h_3*n_{2K}*n_{3K} - h_3**2*n_{1K}**2 - h_3**2*n_{2K}**2 + 2*h_3**2*n_{3K}**2)/(4*h**3)

In [92]:
expr_psi_nk = expr_psi_upsilon_Q0.args[-1]
expr_psi_nk

2*h_1**2*n_{1K}**2 - h_1**2*n_{2K}**2 - h_1**2*n_{3K}**2 + 6*h_1*h_2*n_{1K}*n_{2K} + 6*h_1*h_3*n_{1K}*n_{3K} - h_2**2*n_{1K}**2 + 2*h_2**2*n_{2K}**2 - h_2**2*n_{3K}**2 + 6*h_2*h_3*n_{2K}*n_{3K} - h_3**2*n_{1K}**2 - h_3**2*n_{2K}**2 + 2*h_3**2*n_{3K}**2

In [93]:
expr = expr_psi_nk-(3*(vh.dot(nk)**2)-vh.dot(vh))
expr = expr.expand()
expr = expr.subs({nk1**2: 1-nk2**2-nk3**2})
expr.expand()

0

In [94]:
expr = expr_psi_upsilon_Q0/expr_psi_nk*h**2
expr = expr.expand()

display(expr)
sp.print_latex(expr)

3*\Upsilon*\left<m\right>/(2*h) + {\left<\eta^2\right>}/4 - 1/4

\frac{3 \Upsilon \left<m\right>}{2 h} + \frac{{\left<\eta^2\right>}}{4} - \frac{1}{4}
